In [85]:
#Básicos para manipulacion de datos 
import pandas as pd
import numpy as np
#Graficas 
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
sns.set_theme()
#Optimización multiobjetivo 
from pymoo.util.ref_dirs import get_reference_directions
from pymoo.optimize import minimize
from pymoo.core.population import Population
from pymoo.core.mutation import Mutation

from pymoo.core.population import Population
#Finanzas 
import yfinance as yf
import yesg

#Plugins
from tqdm import tqdm
from itertools import compress
from Plugins import pre_processing
from Plugins import ArchievingStrategies
from Plugins import my_plotting
from Plugins import pymoo_extras
from Plugins import DS

In [86]:
from pymoo.core.sampling import Sampling
class MySampling(Sampling):
    def _do(self, problem, n_samples, **kwargs):
        #100 individuos (a lo más 2000)
        #problem.n_var la cantidad de activos. 
        X = get_reference_directions("energy", problem.n_var, n_samples, seed=1)
        return X 

In [87]:
from pymoo.core.crossover import Crossover
from pymoo.core.variable import Real, get
# ---------------------------------------------------------------------------------------------------------
# Class
# ---------------------------------------------------------------------------------------------------------
def point_on_triangle(x0, x1, x2):
    s, t = sorted([np.random.random(), np.random.random()])
    return s*x0 + (t-s)*x1 + (1-t)*x2
    
def SPX(x0,x1, x2, epsilon,n_offspring): 
    CoM = 1/3*(x0+x1+x2)
    V1=CoM+(x0-CoM)*(1+epsilon)
    V2=CoM+(x1-CoM)*(1+epsilon)
    V3=CoM+(x2-CoM)*(1+epsilon)
    return [point_on_triangle(x0,x1, x2) for _ in range(n_offspring)]

class SimplexCrossover(Crossover):
    def __init__(self,
                 n_offsprings=3, 
                 epsilon = 0.05,
                 **kwargs):
        super().__init__(3, n_offsprings, **kwargs)
        self.epsilon =  epsilon
        
    def _do(self, problem, X, **kwargs):
        _, n_matings, _ = X.shape 
        print(X.shape)
        #print(X[0,:,:])
        #print(X[1, : , :])
        #print(X[2, :, :])
        Y = np.full_like(X, None)
        for k in range(n_matings):
             # get the first and the second parent
            x0, x1, x2 = X[0, k, :], X[1, k, :], X[2, k, :]
            off_ = SPX(x0, x1, x2, self.epsilon, self.n_offsprings)
            Y[0, k, :] = off_[0]
            Y[1, k, :] = off_[1]
            Y[2, k, :] = off_[2]
        #(n_padres, n_matings,n_var)
        #(m_offspings,n_matings,n_var) Idealmente
        return Y


In [88]:
import numpy as np

def hyperplane_truncated_mvnorm(mean, cov, G, r, diag=False, random_state=None):
    rng = np.random.default_rng(random_state)
    if diag:
        cov_diag = np.diag(cov)
        y = mean + cov_diag * rng.standard_normal(mean.shape[0])
        covg = cov_diag[:, None] * G.T
    else:
        y = rng.multivariate_normal(mean, cov, method='cholesky')
        covg = cov @ G.T
    gcovg = G @ covg
    alpha = np.linalg.solve(gcovg, r - G @ y)
    return y + covg @ alpha

In [89]:
import numpy as np
rng = np.random.default_rng()

# generate example input
k1, k2 = 5, 1
temp = rng.random((k1, k1))

cov = temp @ temp.T
G = np.ones((k2, k1))
r = np.ones(k2)
mean = rng.random(k1)

# passing `random_state` is optional. If the argument is not used, a fresh
# random generator state is instantiated internally using system entropy.
o = hyperplane_truncated_mvnorm(mean, cov, G, r, random_state=rng)
print(o.sum())  # verify if sampled values sum to zero
# alternatively one can pass an array to store the results in
hyperplane_truncated_mvnorm(mean, cov, G, r)

0.9999999999999991


array([ 0.47717783, -0.89857061, -0.81290859,  1.113197  ,  1.12110438])

In [111]:
import numpy as np

from pymoo.core.mutation import Mutation
from pymoo.core.variable import Real, get
from pymoo.operators.repair.bounds_repair import repair_random_init

# ---------------------------------------------------------------------------------------------------------
# Function Dirichlet 
# ---------------------------------------------------------------------------------------------------------
def mut_dirichlet(X, xl, xu, prob): 
    n, n_var = X.shape 
    assert len(prob) == n
    Xp = np.full(X.shape, np.inf)
    mut = np.random.random(X.shape) < prob[:, None]
    Xp[:, : ] = X 
    for i in range(n): 
        if mut[i].any(): 
            alphas = (X[i] + 0.05) 
            Xp[i] = np.random.dirichlet(alphas, 1)
    Xp = repair_random_init(Xp, X, xl, xu)
    return Xp
# ---------------------------------------------------------------------------------------------------------
# Function Logit-Normal 
# ---------------------------------------------------------------------------------------------------------
def mut_logit_normal(X, xl, xu, prob): 
    n, n_var = X.shape 
    assert len(prob) == n
    Xp = np.full(X.shape, np.inf)
    mut = np.random.random(X.shape) < prob[:, None]
    Xp[:, : ] = X 
    for i in range(n): 
        if mut[i].any(): 
            mean = X[i][:-1]
            cov  = np.identity(n_var-1)

            norm_vector = np.random.multivariate_normal(mean, cov, 1)
            exp_vector = np.exp(norm_vector)
            sum_exp    = np.sum(exp_vector)
            Xp[i] = logit_norm = np.append( exp_vector/(1+sum_exp), 1/(1+sum_exp))
    Xp = repair_random_init(Xp, X, xl, xu)
    return Xp 

# ---------------------------------------------------------------------------------------------------------
# Function
# ---------------------------------------------------------------------------------------------------------
def mut_gauss(X, xl, xu, prob):
    n, n_var = X.shape
    assert len(prob) == n

    Xp = np.full(X.shape, np.inf)

    mut = np.random.random(X.shape) < prob[:, None] # Solo una vez 

    Xp[:, :] = X
    
    k1, k2 = n_var, 1
    temp = rng.random((k1, k1))
    cov = temp @ temp.T
    G = np.ones((k2, k1))
    r = np.ones(k2)
    for i in range(n): 
        if mut[i].any(): 
            Xp[i] = hyperplane_truncated_mvnorm(X[i], cov, G, r)
    
    Xp = repair_random_init(Xp, X, xl, xu)

    return Xp


# ---------------------------------------------------------------------------------------------------------
# Class
# ---------------------------------------------------------------------------------------------------------


class GaussianMutation(Mutation):

    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def _do(self, problem, X, **kwargs):
        X = X.astype(float)
        prob_var = self.get_prob_var(problem, size=len(X))
        Xp = mut_logit_normal(X, problem.xl, problem.xu, prob_var)

        return Xp

## Experimento

In [91]:
assets = pd.read_csv( 'Indices/tickers_dowjones.csv', index_col=0)['0'].tolist()
returns= pd.read_csv( 'data/dowjones_returns.csv', index_col=0)
assets_info=pd.read_csv( 'data/dowjones_assets_info.csv', index_col=0)
best_assets=pd.read_csv( 'data/dowjones_best_assets.csv', index_col=0)

In [92]:
PROFITS, RISK, ESG_SCORES = pre_processing.get_final_assets(returns[best_assets.index[:5]], assets_info.loc[best_assets.index[:5]])
portfolio_problem = pymoo_extras.Portfolio_Problem(len(PROFITS), PROFITS, RISK, ESG_SCORES)
from finquant import efficient_frontier
ef = efficient_frontier.EfficientFrontier(pd.Series(PROFITS), pd.DataFrame(RISK), freq=252)
ef_R = ef.efficient_frontier()

In [142]:
from pymoo.termination import get_termination
from pymoo.algorithms.moo.nsga2 import NSGA2
termination = get_termination("n_gen", 300)
nsgaii = NSGA2(pop_size=32,
               sampling=MySampling(), 
               crossover=SimplexCrossover(),
               mutation = GaussianMutation(),
               repair=pymoo_extras.Portfolio_Repair()
               )

In [140]:
eps = np.array([0.01,0.01])
colors = ['rgb(27,158, 119)']

In [143]:
X_nsgaii, F_nsgaii, ESG_nsgaii =pymoo_extras.get_weights_with_pymoo(portfolio_problem, nsgaii, termination)
FA_nsgaii =  pymoo_extras.annualised_portfolio_quantities(F_nsgaii)
frames = [FA_nsgaii]
labels = ['NSGA-II']
fig=my_plotting.plotting_samples_plotly(ef_R, frames, labels, colors)
fig.show()

(3, 11, 5)
(3, 11, 5)
(3, 1, 5)
(3, 11, 5)
(3, 1, 5)
(3, 11, 5)
(3, 1, 5)
(3, 11, 5)
(3, 11, 5)
(3, 11, 5)
(3, 11, 5)
(3, 11, 5)
(3, 11, 5)
(3, 11, 5)
(3, 11, 5)
(3, 11, 5)
(3, 11, 5)
(3, 1, 5)
(3, 11, 5)
(3, 11, 5)
(3, 11, 5)
(3, 11, 5)
(3, 11, 5)
(3, 2, 5)
(3, 11, 5)
(3, 11, 5)
(3, 11, 5)
(3, 11, 5)
(3, 1, 5)
(3, 11, 5)
(3, 11, 5)
(3, 11, 5)
(3, 1, 5)
(3, 11, 5)
(3, 11, 5)
(3, 11, 5)
(3, 11, 5)
(3, 11, 5)
(3, 11, 5)
(3, 1, 5)
(3, 11, 5)
(3, 1, 5)
(3, 11, 5)
(3, 11, 5)
(3, 1, 5)
(3, 11, 5)
(3, 11, 5)
(3, 1, 5)
(3, 11, 5)
(3, 1, 5)
(3, 11, 5)
(3, 1, 5)
(3, 11, 5)
(3, 11, 5)
(3, 11, 5)
(3, 1, 5)
(3, 11, 5)
(3, 11, 5)
(3, 1, 5)
(3, 11, 5)
(3, 11, 5)
(3, 11, 5)
(3, 1, 5)
(3, 11, 5)
(3, 11, 5)
(3, 1, 5)
(3, 11, 5)
(3, 11, 5)
(3, 11, 5)
(3, 1, 5)
(3, 11, 5)
(3, 11, 5)
(3, 11, 5)
(3, 1, 5)
(3, 11, 5)
(3, 11, 5)
(3, 11, 5)
(3, 11, 5)
(3, 11, 5)
(3, 11, 5)
(3, 1, 5)
(3, 11, 5)
(3, 11, 5)
(3, 11, 5)
(3, 1, 5)
(3, 11, 5)
(3, 11, 5)
(3, 11, 5)
(3, 2, 5)
(3, 11, 5)
(3, 11, 5)
(3, 1, 5)
(3, 11, 5)


In [144]:
# Algoritmo | Problem | Ejecucion | Generacion(Iteracion) | HV  (Poblacion al momento)  | HV (archivo que almacena los optimos)
#   NSGA-II     DTLZ1       1               1                0.83  (32)
#                          1               2                0.94  (32)        
# NSGA-II Cruza-1
# NSGA-II Cruza-2 
# NSGA-II Mutaciones 
# NSGA-II Cruza-Mutacion
#   GridSearch espacio de diseño del algoritmo BASE DE DATOS
 #¿Para qué guardo a X? 
#Arca

#Correr todos con todos los operadores  #De momento solo sobre última generacion
#20 ejecuciones  (Series de tiempo) ¿Es suficiente - investigar si hay que hacer más?***
#Generacion = 250  
#Poblacion  = 100

#Normalizar sobre todos los resultados OOO Normalizar por algoritmo , sacar HV solo a la última generación 
#Evaluar HV por generacion  
#Evaluar HV óptimos 


#Número de variables = 12
#DTLZ con 3 objetivos 12 variables  (7)
#ZDT con 2 objetivos, 30 variables  (ZDT4 - 10 variables ) (6)
#WFG con 3 objetivos y 24 variables (9)


#1 Ejecución con cada problema  (Imagen) y la población óptimos 

In [ ]:
#Abstract 
#Introduccion ¿a qué? 
#Trabajos relacionados, estado del arte 
#Algoritmo Evolutivo  - Cambiando operadores 
#Problemas 
#Aplicación de Portafolio 3 objetivos ESG 
#ocupo el evolutivo para dos objetivos 
#ocupo PQ-eps para filtrar en 3 objetivos. 
#Conclusiones 
